# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
import os
from utils import get_openai_api_key, get_serper_api_key

openai_api_key = get_openai_api_key()
os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
os.environ["SERPER_API_KEY"] = get_serper_api_key()

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool

search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically. ~ set to over look all and iterate if anything is required
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    manager_llm=ChatOpenAI(model="gpt-3.5-turbo", 
                           temperature=0.7),
    process=Process.hierarchical,
    verbose=True
)

## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

In [15]:
financial_trading_inputs1 = {
    'stock_selection': 'NVDA',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [16]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of continuously monitoring and analyzing market data for AAPL to the Data Analyst to gather insights and alerts about significant market opportunities or threats for AAPL.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for AAPL using statistical modeling and machine learning", "context": "The goal is to identify trends and predict market movements for AAPL."}

> Entering new CrewAgentExecutor chain...
I need to gather real-time market data for AAPL to analyze trends and predict market movements.

Action: Read website content
Action Input: {"website_url": "https://finance.yahoo.com/quote/AAPL"} 

Apple 

Final Answer: Based on the real-time market data from Yahoo Finance for AAPL, the stock price of Apple Inc. (AAPL) has increased by 7.26% to $207.15. The company's market cap is $3.176 trillion, and it operates in the consumer electronics and technology sector. Key financial highlights include a profit margin of 26.31%, return on equity of 147.25%, revenue of $381.62 billion, and net income of $100.39 billion. Additionally, the company has a forward P/E ratio of 26.25 and a PEG ratio of 2.04. This data can be used to analyze trends and predict market movements for AAPL.

> Finished chain.
 

Based on the real-time market data from Yahoo Finance for AAPL, the stock price of Apple Inc. (AAPL) has increased by 7.26% to $207.15. The company's market cap is $3.176 trillion, and it operates in the consumer electronics and technology sector. Key financial highlights include a profit margin of 26.31%, return on equity of 147.25%, revenue of $381.62 billion, and net income of $100.39 billion. A

Final Answer: 
Based on the real-time market data from Yahoo Finance, AAPL stock price has increased by 7.26% to $207.15. The company's market cap is $3.176 trillion, profit margin of 26.31%, return on equity of 147.25%, revenue of $381.62 billion, net income of $100.39 billion, forward P/E ratio of 26.25, and PEG ratio of 2.04. Apple Inc. (AAPL) designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, Mac, iPad, and wearables, among other products. Apple also provides various services such as Apple Arcade, Apple Fitness+, Apple Music, Apple News+, Apple TV+, and Apple Card. The company serves consumers, small and mid-sized businesses, and the education, enterprise, and government markets. Apple Inc. was founded in 1976 and is headquartered in Cupertino, California.

> Finished chain.
 

Based on the real-time market data from Yahoo Finance, AAPL stock price has increased by 7.26% to $207.15. The compa

Final Answer: 
After conducting a search on approved trading strategies for AAPL, I found several helpful resources:

1. AAPL Option Strategy Payout Diagram Apple - Market Chameleon
Link: https://marketchameleon.com/Overview/AAPL/StrategyPayout/
Snippet: Analyze Apple (AAPL) stock option trading strategies. Display payout diagrams showing gains and losses for Straddle, Buy-Write, Risk Reversal, Call Spread, ...

2. Trading Strategies For Apple Stock Heading Into Q4 Earnings
Link: https://markets.businessinsider.com/news/etf/trading-strategies-for-apple-stock-heading-into-q4-earnings-1032771973
Snippet: Traders wanting to play the potential upside in Apple may choose to take a position in Direxion Daily Apple Bull 1.5X Shares (NASDAQ:AAPU), ...

3. Three Day Trading Strategies for Apple Stocks - AskTraders.com
Link: https://www.asktraders.com/learn-to-trade/trading-guide/three-day-trading-strategies-apple-stocks/
Snippet: Three Day Trading Strategies for Apple Stocks · 1. Two-day high/l

I need to gather detailed information on the proposed trading strategies for AAPL to evaluate the associated risks accurately.

Action: Read website content
Action Input: {"website_url": "AAPLtradingstrategies.com"} 

I encountered an error while trying to use the tool. This was the error: Invalid URL 'AAPLtradingstrategies.com': No scheme supplied. Perhaps you meant https://AAPLtradingstrategies.com?.
 Tool Read website content accepts these inputs: Read website content(website_url: 'string') - A tool that can be used to read a website content.

Thought: 
I need to gather detailed information on the proposed trading strategies for AAPL to evaluate the associated risks accurately.

Action: Search the internet
Action Input: {"search_query": "AAPL trading strategies"} 


Search results: Title: AAPL Option Strategy Payout Diagram Apple - Market Chameleon
Link: https://marketchameleon.com/Overview/AAPL/StrategyPayout/
Snippet: Analyze Apple (AAPL) stock option trading strategies. Display p

Thought: The Barchart website provides trading strategies and performance for AAPL.
Action: Read website content
Action Input: {"website_url": "https://www.barchart.com/stocks/quotes/AAPL/trading-strategies"} 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.



Final Answer: 
After conducting a search on the internet, I found several resources that provide information on trading strategies for AAPL. These strategies include options trading, sector rotation, and earnings-driven strategies. The Barchart website offers insights into these strategies, along with performance data and recommendations for AAPL stock. By incorporating elements of each trading strategy and adapting them to fit AAPL's specific market dynamics, traders can aim to maximize returns while managing risks effectively. It is crucial for traders to monitor market trends, financial performance, and sector correlations to make informed decisions on trading AAPL stock.


In [17]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs1)

 [DEBUG]: Working Agent: Crew Manager
 [INFO]: Starting Task: Continuously monitor and analyze market data for the selected stock (NVDA). Use statistical modeling and machine learning to identify trends and predict market movements.


> Entering new CrewAgentExecutor chain...
I need to delegate the task of continuously monitoring and analyzing market data for the selected stock (NVDA) to the Data Analyst in order to identify trends and predict market movements.

Action: Delegate work to co-worker
Action Input: {"coworker": "Data Analyst", "task": "Continuously monitor and analyze market data for NVDA", "context": "The goal is to use statistical modeling and machine learning to identify trends and predict market movements for NVDA."}

> Entering new CrewAgentExecutor chain...
I need to gather the latest market data for NVDA to analyze trends and make predictions.

Action: Read website content
Action Input: {"website_url": "https://finance.yahoo.com/quote/NVDA"} 

NVIDIA Corporation (NVD

Final Answer: NVIDIA Corporation provides graphics and compute and networking solutions in the United States, Taiwan, China, Hong Kong, and internationally. The company's products are used in gaming, professional visualization, data center, and automotive markets. It sells its products to various entities in the technology sector. NVDA stock has experienced significant growth over different time periods, indicating strong performance in the market.

> Finished chain.
 

NVIDIA Corporation provides graphics and compute and networking solutions in the United States, Taiwan, China, Hong Kong, and internationally. The company's products are used in gaming, professional visualization, data center, and automotive markets. It sells its products to various entities in the technology sector. NVDA stock has experienced significant growth over different time periods, indicating strong performance in the market.

Final Answer: 

As the Data Analyst, your task is to continuously monitor and analyze

Final Answer: 
NVIDIA Corporation (NVDA) Stock Price, News, Quote & History - Yahoo Finance
 News Today's news US Politics World Tech Reviews and deals Audio Computing Gaming Health Home Phones Science TVs Climate change Health Science 2024 election Originals The 360 Life Health COVID-19 Fall allergies Health news Mental health Relax Sexual health Studies The Unwind Parenting Family health So mini ways Style and beauty It Figures Unapologetically Horoscopes Shopping Buying guides Food Travel Autos Gift ideas Buying guides Entertainment Celebrity TV Movies Music How to Watch Interviews Videos Shopping Finance My Portfolio News Latest News Stock Market Originals Premium News Economics Housing Earnings Tech Crypto Biden Economy Markets Stocks: Most Actives Stocks: Gainers Stocks: Losers Trending Tickers Futures World Indices US Treasury Bonds Rates Currencies Crypto Top ETFs Top Mutual Funds Options: Highest Open Interest Options: Highest Implied Volatility Sectors Basic Materials Communi

Final Answer: 
After delegating the task to the Data Analyst to continuously monitor and analyze market data for NVDA, identify trends, and predict market movements using statistical modeling and machine learning, the Trading Strategy Developer can use these insights to develop and refine trading strategies for NVDA. By aligning the strategies with the user's risk tolerance and considering trading preferences such as day trading, we can create a set of potential trading strategies for NVDA that will guide strategic decisions for our team. This collaborative effort will ensure that the trading strategies are well-informed, data-driven, and tailored to meet the user's risk tolerance while maximizing opportunities in the market.

> Finished chain.
 [DEBUG]: [Crew Manager] Task output: After delegating the task to the Data Analyst to continuously monitor and analyze market data for NVDA, identify trends, and predict market movements using statistical modeling and machine learning, the Trad

Final Answer: NVIDIA Corporation (NVDA) provides graphics and compute and networking solutions in the United States, Taiwan, China, Hong Kong, and internationally. The company offers GeForce GPUs for gaming and PCs, the GeForce NOW game streaming service, Quadro/NVIDIA RTX GPUs for enterprise workstation graphics, virtual GPU software for cloud-based visual computing, automotive platforms for infotainment systems, and Omniverse software for 3D internet applications. The Compute & Networking segment includes Data Center computing platforms, NVIDIA DRIVE automated-driving platform, Jetson robotics platforms, NVIDIA AI Enterprise software, and DGX Cloud services. NVDA's products are used in gaming, professional visualization, data center, and automotive markets. The stock price for NVDA closed at 120.91 on June 11, with a YTD return of 144.15% and a 1-year return of 211.86%. The company has a market cap of 2.974T and employs 29,600 full-time employees.

> Finished chain.
 

NVIDIA Corpora

Action Input: {"search_query": "NVDA trading strategies and risks"} 


Search results: Title: 4 Risks Nvidia Investors Should Consider as They Celebrate the ...
Link: https://www.nasdaq.com/articles/4-risks-nvidia-investors-should-consider-they-celebrate-nvda-stock-split
Snippet: Nvidia (NVDA) has started trading today on a split-adjusted basis. Investors have a lot to celebrate, as Nvidia has been a stellar wealth ...
---
Title: A Way To Go Bullish On Nvidia Earnings With Limited Risk
Link: https://www.investors.com/research/options/nvidia-stock-earnings-bullish-trade/
Snippet: All eyes will be on Nvidia stock earnings next week. This bull put spread option trade offers a nice profit with limited risk.
---
Title: NVDA Option Trade Could Unlock a 36% Annualized Profit
Link: https://www.theglobeandmail.com/investing/markets/indices/TXCX/pressreleases/26771999/nvda-option-trade-could-unlock-a-36-annualized-profit/
Snippet: - If NVDA stock trades sideways and finishes between $115 and $14

- Display the final result as Markdown.

In [18]:
from IPython.display import Markdown
Markdown(result)

Risk Analysis Report for NVDA Trading Strategies:

Introduction:
The following risk analysis report aims to identify potential risks associated with the proposed trading strategies and execution plans for NVDA. This analysis will provide detailed insights into key risk factors that could impact NVDA's stock price and offer mitigation recommendations to manage these risks effectively.

Key Risks:
1. Stock Splits and Corporate Activity:
Nvidia investors should consider the potential risks related to stock splits and corporate activities. Any significant changes in Nvidia's corporate structure, such as mergers, acquisitions, or divestitures, could impact the stock price and investor sentiment. It is essential for traders and investors to stay informed about any upcoming corporate actions and assess their potential impact on NVDA's stock performance.

2. Earnings Volatility:
Trading Nvidia stock before and after Q4 earnings involves specific risks that traders and investors should be aware of. Earnings reports can lead to significant price fluctuations in NVDA's stock, depending on whether the results meet, exceed, or fall short of market expectations. Traders should exercise caution when trading around earnings announcements and consider implementing risk management strategies to protect their investments.

3. Market Fluctuations and Valuation:
With Nvidia's stock valuation reaching $2 trillion, there are inherent risks that investors should consider, especially in terms of market fluctuations and volatility. Changes in market conditions, macroeconomic factors, and industry trends can impact NVDA's stock price and overall performance. Traders should monitor market developments closely and adjust their trading strategies accordingly to mitigate potential losses.

Mitigation Recommendations:
To mitigate the risks associated with NVDA trading strategies, traders and investors should consider the following mitigation recommendations:

1. Diversification: Diversifying the investment portfolio across different asset classes, sectors, and industries can help spread risk and reduce exposure to individual stock fluctuations. By diversifying, traders can protect their investments from the impact of specific events affecting NVDA's stock price.

2. Risk Management Strategies: Implementing risk management strategies such as stop-loss orders, setting profit targets, and using options strategies like covered calls and protective puts can help mitigate potential losses and protect investment capital. Traders should establish clear risk management rules and adhere to them consistently to manage risk effectively.

3. Market Research: Conducting thorough research on NVDA, market trends, and industry developments is essential for making informed trading decisions. Traders should stay informed about company-specific news, economic indicators, and market sentiment to anticipate potential risks and opportunities in the market.

Conclusion:
In conclusion, the comprehensive risk analysis report highlights key risks associated with NVDA trading strategies and provides mitigation recommendations to manage these risks effectively. By identifying and addressing potential risk factors, traders and investors can make informed decisions, protect their investments, and capitalize on the opportunities presented by the market. It is crucial for the team to align their trading strategies with the firm's risk tolerance and adjust their approach as needed based on changing market conditions.